# Aerodynamic Drag 1 - Components of Drag

This notebook explores how we can breakdown the complex process of drag estimation, into more manageable actions.

> Please remember I am not an authority on drag analysis, but this is my best interpretation of what's going on. If you spot any mistakes, please do reach out!

    ~ Yaseen

## Modelling Formulation

The force of drag acting on an aircraft can be broken down into linear contributions from many sources

$$D = \Sigma_{j=0}^{N} D_j \qquad \text{with} \qquad N >> 1$$

It's helpful to non-dimensionalise the drag force into a linear sum of coefficients of drag

$$D = \frac{1}{2}\rho V^2 S C_D \qquad \text{where} \qquad C_D = \Sigma_{j=0}^N C_{D_{j}}$$

and now, we can break up this drag coefficient $C_D$ into one of the following categories

\begin{eqnarray}
C_D &=& C_{D_0} &+& C_{D_i} &+& C_{D_w} \\
\uparrow && \uparrow && \uparrow && \uparrow \\
\text{Total Drag} &=& \text{Profile Drag} &+& \text{Induced Drag} &+& \text{Wave Drag}
\end{eqnarray}

We'll explore contributions to these categories momentarily, but it's important to understand that most contributions to these categories can themselves be broken into two subcategories for the purposes of effectual mathematical modelling:
    
1. **Drag due to Lift**:
    These are components of drag that arise as a result of a vehicle needing to produce a lift force. If an aerodynamic body is not producing lift, the contributions from these components are negligible.

1. **Zero-lift drag**:
    What doesn't fall into the subcategory of lift-dependent drag are simply those drag subcomponents that are independent of lift. The magnitude of non-dimensional drag coefficients is non-zero and contribute to $C_{D}$ - even when the aerodynamic object is producing no lift.

In most introductory lessons to drag, you'll find a slightly different representations of the drag breakdown using similar symbology and scripting.
We'll clarify two common portrayals below by first recapping what this means for the formulation above:

\begin{eqnarray}
\text{Profile Drag } (C_{D_0}) &=& \text{Zero-lift drag components} &+& \text{(Lift-) Induced drag components}\\
\text{Induced Drag } (C_{D_i}) &=& 0 &+& \text{(Lift-) Induced drag components}\\
\text{Wave Drag } (C_{D_w}) &=& \text{Zero-lift drag components} &+& \text{(Lift-) Induced drag components}
\end{eqnarray}

and in the second model, we sum all the *zero-lift* and *lift-induced* drag subcomponents across all the categories:

\begin{eqnarray}
\text{Minimum Drag } (C_{D_{\text{min}}}) &\equiv& \text{Zero-lift Drag } (C_{D_0}) &=& \text{Zero-lift drag from} & \text{[Profile + Wave]}\\
&& \text{Induced Drag } (C_{D_i}) &=& \text{Induced drag from} & \text{[Profile + Lift-Induced Drag + Wave]}
\end{eqnarray}

In both representations, you will often see $C_{D_0} = \text{constant}$ and $C_{D_i} = f(C_L)$.
In reality one should see that for constant $C_{D_0}$, the first model would have to neglect lift-induced contributions to profile drag, and the second model would neglect the zero-lift components from wave drag that only arise after the aerodynamic body encounters transonic and supersonic flow effects.

This often makes the second representation more prevalent in that introductory drag modelling first considers simpler subsonic cases, and not much thought is put into estimating the lift-induced subcomponent of profile drag.

*So which representation should you care about for this library?*

We'll stick to the first formulation because it's simpler to discretise in this library's architecture.
*Profile drag* estimation tends to be a 2D problem for 2D geometry, *Induced drag* can be estimated as part of the mapping from 2D aerofoils to 3D wings, and *Wave drag* only makes sense to compute after major 3D geometry of a vehicle has already been established.

## Profile Drag, $C_{D_0}$

Profile drag can largely be thought of as the drag associated with the 2D profile of aerofoils.
Profile drag is also sometimes known as *parasitic drag* (an ever present drag component sapping away at your aircraft's hard earned energy state).

#### *Zero-lift* component

- Pressure drag, a.k.a Form drag

Pressure (form) drag is the drag associated with the separation of flow from a surface.
The resulting turbulence and eddy currents in the wake flow of an object are responsible for a suction-type drag effect.
Form drag is a function of how slender or streamlined an object is.

In addition to this, the mixing of dissimilar airflows causes *interference* drag.
This is a type of form drag in which turbulence and eddy currents in the wake flow of an object, as you might've guessed by now, *also* create a suction-type drag effect.

Air molecules do not care that a weapons pylon is a small component jutting out of a larger wing, or that the nose gear is a separate element that sticks out of the fuselage - the molecules encounter "*the vehicle*" just the same - some flow separates around some geometry, and other flows mix poorly around others (both causing form drag).

- Skin friction drag

Thanks to the *no-slip boundary condition*, particles in viscous fluids closest to a solid boundary have no velocity relative to the boundary - i.e. the forces of adhesion to the wall are stronger than the forces of cohesion with other particles closer to the freestream.
The size of the boundary layer, how laminar or turbulent it is, and the viscosity of the fluid all play important roles in being able to estimate the frictional drag force caused by the aerodynamically *wetted* skin of the object.

Skin friction drag can be estimated by considering a coefficient of friction somehwere between empirically determined laminar and turbulent solutions.
The position at which oncoming laminar flow transitions to turbulent flow is dictated by the flow Reynolds number, as well as the quality of the aerodynamic surface's finish.

#### *Lift-induced* component

- Skin friction drag

In addition to the smoothness of the aerodynamic surface and Reynolds number, the transition point of a flow from laminar to turbulent is also given by the pressure gradient over an aerofoil (which changes with angle of attack).
A favourable pressure gradient is that in which the pressure gradient is reducing (negative gradient) causing the flow to accelerate and stabilise the boundary layer, allowing for an extensive laminar regime.
Giving the flow too short a distance to decelerate will cause flow separation.
Varying the sectional angle of attack will change the chordwise pressure distribution, impacting transition points on upper and lower surfaces.

## Induced Drag, $C_{D_i}$

Despite the confusing nomenclature, induced drag in this context primarily refers to the drag produced as the by-product of lift production in any lifting surfaces.
This term does not include lift-induced drag subcomponents of Profile or Wave drag.

*By definition, this term has no zero-lift drag component.*

#### *Lift-induced* component

- Pressure drag, a.k.a Vortex drag

Induced drag is a purely 3D phenomena, in that it is caused by the lateral motion of air across a wing.
The higher pressure surfaces below a wing spill pressure energy around the wing tips into the lower pressure zones above a wing.
This causes a swirling vortex of air with an axis of rotation near the wing tips (hence they are called 'tip vortices'), where air below the wing moves outboard, and air above the wing moves inward.
This movement of air is known as 'downwash', induced by the tip vortices.

According to Prandtl's Lifting-line Theory (PLLT), lifting surfaces can be modelled as a continuous sheet of vortices and aptly models this circulatory flow.
Considering for a moment a 2D section of the wing - the downwash induces a local, downward velocity component that reduces the 
section's effective angle of attack $\alpha_{\text{eff}}$ by the induced angle of attack $\alpha_i$.

$$\alpha_{\text{eff}} = \alpha_{\infty} - \alpha_i$$

Despite freestream angle of attack $\alpha_{\infty}$ remaining unchanged, the local reduction in a section's angle of attack has the effect of tilting the lift vector backwards (towards the trailing edge) by $\alpha_i$.
This rotation of this local, sectional lift vector means less of it is contributing to the wing's lift force normal to the freestream direction we care about - and even worse, the remaining component of the vector now acts in a direction opposing motion of the wing (hence 'lift-induced' drag).

## Wave Drag, $C_{D_w}$ --- \**Coming soon*$^{\text{TM}}$\*

#### *Zero-lift* component

- Wave drag due to volume

#### *Lift-induced* component

- Wave drag due to lift